# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-NOTEBOOK_CODE_HERE-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Book Workflow Series ("Authors comparison explorative search") 

Consider the following exploratory scenario:


>  Investigate the the production of Paul Auster and Ian McEwan, check how many books they have written for each litarature genre, gather information about their production and about their works which are not books



## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wd:Q47461344`| writtenwork   | node |
| `wd:Q214642`  | Paul Auster   | node |
| `wd:Q190379`  | Ian McEwan    | node |




Also consider

```
wd:Q214642 ?rel ?obj  . 
```
is the BGP to retrieve all **properties of Paul Auster**


The workload should


1. Identify the BGP for obtaining the books (with publishing date and genre) published by the two authors

2. Did the authors published a book in the same year? What is the longest period without publishing a book for the two authors? 

3. Did the authors produced, acted or directed a film? If so, did they write the screenplay?

4. How many films were derived from the books of these two authors? 

5. Which author won more literature-related awards? Have they ever being nominated for a Nobel award? 

In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT( ?obj)
WHERE { 
wd:Q214642 ?rel ?obj  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '191')]


1

### 1. Identify the BGP for obtaining the books (with publishing date and genre) published by the two authors

Let's analyze the links to the authors

In [4]:
queryString = """
SELECT ?p ?pname ?o ?oname WHERE { 
    ?authors ?p ?o  .
 
    # this returns the labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    
    FILTER ((?authors = wd:Q214642) || (?authors = wd:Q190379)) .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('pname', 'spouse'), ('o', 'http://www.wikidata.org/entity/Q1280035'), ('oname', 'Lydia Davis')]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pname', 'notable work'), ('o', 'http://www.wikidata.org/entity/Q581485'), ('oname', 'The Book of Illusions')]
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('pname', 'family name'), ('o', 'http://www.wikidata.org/entity/Q37565995'), ('oname', 'McEwan')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at'), ('o', 'http://www.wikidata.org/entity/Q1045828'), ('oname', 'University of East Anglia')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('pname', 'given name'), ('o', 'http://www.wikidata.org/entity/Q18336315'), ('oname', 'Ian')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at'), ('o', 'http://www.wikidata.org/entity/Q1161297'), ('oname', 'University of Sussex')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pn

98

Since it's complicated that the books are all listed as links to the authors, let's try by searching books written by the authors

In [8]:
queryString = """
SELECT ?b ?bname ?p ?pname WHERE { 
    ?b ?p ?authors.
 
    # this returns the labels
    ?p <http://schema.org/name> ?pname .
    ?b <http://schema.org/name> ?bname .
    
    FILTER ((?authors = wd:Q214642) || (?authors = wd:Q190379)) .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('b', 'http://www.wikidata.org/entity/Q160082'), ('bname', 'Booker Prize'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
[('b', 'http://www.wikidata.org/entity/Q5709276'), ('bname', 'Helmerich Award'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
[('b', 'http://www.wikidata.org/entity/Q2915460'), ('bname', 'Act of God'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]
[('b', 'http://www.wikidata.org/entity/Q2364684'), ('bname', 'The Music of Chance'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]
[('b', 'http://www.wikidata.org/entity/Q7348342'), ('bname', 'The Unbelievers'), ('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]
[('b', 'http://www.wikidata.org/entity/Q3213673'), ('bname', 'The Inner Life of Martin Frost'), ('p', 'http://www.wikidata.org/prop/direct/P162'), ('pname', 'producer')]
[('b', 'http://www.wikidata.org/entity/Q16261

50

In [9]:
queryString = """
SELECT ?p ?pname ?o ?oname WHERE { 
     wd:Q581485 ?p ?o.
 
    # this returns the labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P50'), ('pname', 'author'), ('o', 'http://www.wikidata.org/entity/Q214642'), ('oname', 'Paul Auster')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows'), ('o', 'http://www.wikidata.org/entity/Q492991'), ('oname', 'Timbuktu')]
[('p', 'http://www.wikidata.org/prop/direct/P407'), ('pname', 'language of work or name'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('oname', 'English')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pname', 'country of origin'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oname', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q47461344'), ('oname', 'written work')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q8261'), ('oname', 'novel')]
[('p', 'http://www.wikidata.org/prop/direct/P840'), ('pname', 'narrative loca

10

P50 (author) and P136(genre) is what I need

In [9]:
queryString = """
SELECT ?books ?genre WHERE { 
    ?s wdt:P50 ?authors ;
        wdt:P136 ?g .
 
    # this returns the labels
    ?s <http://schema.org/name> ?books .
    ?g <http://schema.org/name> ?genre .

    
    FILTER ((?authors = wd:Q214642) || (?authors = wd:Q190379)) .
}
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('books', 'Moon Palace'), ('genre', 'picaresque novel')]
[('books', 'Winter Journal'), ('genre', 'memoir')]
[('books', 'The Invention of Solitude'), ('genre', 'memoir')]
[('books', 'Report from the Interior'), ('genre', 'memoir')]
[('books', 'Leviathan'), ('genre', 'crime novel')]
[('books', 'Squeeze Play'), ('genre', 'crime novel')]
[('books', 'The Invention of Solitude'), ('genre', 'autobiography')]
[('books', 'Mr. Vertigo'), ('genre', 'novel')]
[('books', 'Timbuktu'), ('genre', 'novel')]
[('books', 'The Book of Illusions'), ('genre', 'novel')]
[('books', 'Travels in the Scriptorium'), ('genre', 'novel')]
[('books', 'Saturday'), ('genre', 'novel')]
[('books', 'Man in the Dark'), ('genre', 'novel')]
[('books', 'Sunset Park'), ('genre', 'novel')]
[('books', 'Invisible'), ('genre', 'novel')]
[('books', 'The New York Trilogy'), ('genre', 'novel')]
[('books', 'The Locked Room'), ('genre', 'novel')]
[('books', 'Winter Journal'), ('genre', 'novel')]
[('books', 'The Story of My Type

50

I can't find date of publications on properties of the book. I search it in another book, found on notable work on first query

In [ ]:
queryString = """
SELECT ?pname ?sname WHERE { 
     wd:Q1198186 ?p ?s.
 
    # this returns the labels
    ?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
    
} 
"""

print("Results")
run_query(queryString)

Results
[('pname', 'main subject'), ('sname', 'orphan')]
[('pname', 'main subject'), ('sname', 'death')]
[('pname', 'country of origin'), ('sname', 'United Kingdom')]
[('pname', 'language of work or name'), ('sname', 'English')]
[('pname', 'instance of'), ('sname', 'written work')]
[('pname', 'genre'), ('sname', 'novel')]
[('pname', 'author'), ('sname', 'Ian McEwan')]
[('pname', 'publisher'), ('sname', 'Jonathan Cape')]
[('pname', 'main subject'), ('sname', 'puberty')]
[('pname', 'derivative work'), ('sname', 'The Cement Garden')]


10

### 2. Did the authors published a book in the same year? What is the longest period without publishing a book for the two authors?

Cannot find any data about the publication date of the books

### 3. Did the authors produced, acted or directed a film? If so, did they write the screenplay

In [9]:
I start with Ian McEwan.

SyntaxError: invalid syntax (<ipython-input-9-64eb545e35af>, line 1)

In [38]:
queryString = """

SELECT ?p ?pname ?o ?oname WHERE { 
    wd:Q214642 ?p ?o .
 
    # this returns the labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('pname', 'spouse'), ('o', 'http://www.wikidata.org/entity/Q1280035'), ('oname', 'Lydia Davis')]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pname', 'notable work'), ('o', 'http://www.wikidata.org/entity/Q581485'), ('oname', 'The Book of Illusions')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth'), ('o', 'http://www.wikidata.org/entity/Q25395'), ('oname', 'Newark')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of'), ('o', 'http://www.wikidata.org/entity/Q7118978'), ('oname', 'PEN America')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q11774202'), ('oname', 'essayist')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received'), ('o', 'http://www.wikidata.org/entity/Q13452531'), ('oname', 'Commandeur des Arts et des Lettres\u200e')]
[('p', 'http://www.wikidata.org/prop/dire

44

Let's now find the code of movies
title of movies that Ian McEwan directed.

In [49]:
queryString = """
SELECT ?p ?pname ?o ?oname WHERE { 
    wd:Q2526255 ?p ?o .
 
    # this returns the labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q4220920'), ('oname', 'filmmaking occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q28640'), ('oname', 'profession')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q3455803'), ('oname', 'director')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from'), ('o', 'http://www.wikidata.org/entity/Q222344'), ('oname', 'cinematographer')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q3919997'), ('oname', 'Category:Film directors')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q3391743'), ('oname', 'visual artist')]
[('p', 'http://www.wikidata.org/pro

12

In [50]:
queryString = """
SELECT ?p ?pname ?o ?oname WHERE { 
    wd:Q15488100 ?p ?o .
 
    # this returns the labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q62302889'), ('oname', 'art practice')]
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('pname', 'practiced by'), ('o', 'http://www.wikidata.org/entity/Q2526255'), ('oname', 'film director')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q4671286'), ('oname', 'academic major')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of'), ('o', 'http://www.wikidata.org/entity/Q932586'), ('oname', 'filmmaking')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q1914636'), ('oname', 'activity')]


5

In [51]:
queryString = """
SELECT ?p ?pname ?o ?oname WHERE { 
    wd:Q932586 ?p ?o .
 
    # this returns the labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q21198548'), ('oname', 'audiovisual production')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pname', 'product or material produced'), ('o', 'http://www.wikidata.org/entity/Q11424'), ('oname', 'film')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q11862829'), ('oname', 'academic discipline')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q2657718'), ('oname', 'Armenian Soviet Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('pname', 'practiced by'), ('o', 'http://www.wikidata.org/entity/Q3282637'), ('oname', 'film producer')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q590870'), ('oname', 'cinematography')]
[('p', 

14

Finally, I find that movies are Q11424. Now I need to know how titles of movies are related with it.

In [54]:
queryString = """
SELECT ?p ?pname ?o ?oname WHERE { 
    wd:Q11424 ?p ?o .
 
    # this returns the labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2579'), ('pname', 'studied by'), ('o', 'http://www.wikidata.org/entity/Q1660187'), ('oname', 'film studies')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P345'), ('oname', 'IMDb ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1040'), ('oname', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P344'), ('oname', 'director of photography')]
[('p', 'http://www.wikidata.org/prop/direct/P5869'), ('pname', 'model item'), ('o', 'http://www.wikidata.org/entity/Q17738'), ('oname', 'Star Wars: Episode IV – A New Hope')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q55321'), ('oname', 'Category:Films

10

In [ ]:
I use 'Star Wars: Episode IV – A New Hope' as a template.

In [57]:
queryString = """
SELECT ?p ?pname ?o ?oname WHERE { 
    wd:Q17738 ?p ?o .
 
    # this returns the labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q1351454'), ('oname', 'William Hootkins')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q210447'), ('oname', 'Mark Hamill')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pname', 'film editor'), ('o', 'http://www.wikidata.org/entity/Q38222'), ('oname', 'George Lucas')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('pname', 'producer'), ('o', 'http://www.wikidata.org/entity/Q38222'), ('oname', 'George Lucas')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('pname', 'director'), ('o', 'http://www.wikidata.org/entity/Q38222'), ('oname', 'George Lucas')]
[('p', 'http://www.wikidata.org/prop/direct/P58'), ('pname', 'screenwriter'), ('o', 'http://www.wikidata.org/entity/Q38222'), ('oname', 'George Lucas')]
[('p', 'http://www.wikidata.org/prop/direct/P1431'), ('pname', 'executive 

113

In [ ]:
So returning to the original query, I use P57

In [60]:
queryString = """
SELECT ?s ?filmdirected WHERE { 
    ?s wdt:P57 wd:Q190379 .
 
    # this returns the labels
    ?s <http://schema.org/name> ?filmdirected .
} 
"""

print("Results")
run_query(queryString)

Results
Empty


0

He doesn't direct any movie. Let's find if he's been an actor (P161) or a screenwriter(P58).

In [10]:
queryString = """
SELECT ?s ?film WHERE { 
    ?s wdt:P161 wd:Q190379 .
 
    # this returns the labels
    ?s <http://schema.org/name> ?film .
} 
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q7348342'), ('film', 'The Unbelievers')]


1

Now let's show the movie for which he has been the screenwriter

In [67]:
queryString = """
SELECT ?s ?movie WHERE { 
    ?s wdt:P58 wd:Q190379 .
 
    # this returns the labels
    ?s <http://schema.org/name> ?movie .
} 
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q589713'), ('movie', 'The Good Son')]
[('s', 'http://www.wikidata.org/entity/Q129813'), ('movie', 'Enduring Love')]
[('s', 'http://www.wikidata.org/entity/Q7757528'), ('movie', "The Ploughman's Lunch")]
[('s', 'http://www.wikidata.org/entity/Q1879989'), ('movie', 'The Comfort of Strangers')]
[('s', 'http://www.wikidata.org/entity/Q27959336'), ('movie', 'On Chesil Beach')]
[('s', 'http://www.wikidata.org/entity/Q1626186'), ('movie', 'Atonement')]
[('s', 'http://www.wikidata.org/entity/Q1916423'), ('movie', 'The Cement Garden')]
[('s', 'http://www.wikidata.org/entity/Q426397'), ('movie', 'The Innocent')]
[('s', 'http://www.wikidata.org/entity/Q28923729'), ('movie', 'The Children Act')]


9

Now let's search about Paul Auster.
Firstly, director.

In [69]:
 queryString = """
SELECT ?s ?sname WHERE { 
    ?s wdt:P57 wd:Q214642 .
 
    # this returns the labels
    ?s <http://schema.org/name> ?sname .
} 
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q2478977'), ('sname', 'Blue in the Face')]
[('s', 'http://www.wikidata.org/entity/Q3213673'), ('sname', 'The Inner Life of Martin Frost')]
[('s', 'http://www.wikidata.org/entity/Q653447'), ('sname', 'Smoke')]
[('s', 'http://www.wikidata.org/entity/Q1877215'), ('sname', 'Lulu on the Bridge')]


4

Actor

In [70]:
 queryString = """
SELECT ?s ?sname WHERE { 
    ?s wdt:P161 wd:Q214642 .
 
    # this returns the labels
    ?s <http://schema.org/name> ?sname .
} 
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q2915460'), ('sname', 'Act of God')]
[('s', 'http://www.wikidata.org/entity/Q2364684'), ('sname', 'The Music of Chance')]


2

Screenwriter

In [71]:
 queryString = """
SELECT ?s ?sname WHERE { 
    ?s wdt:P58 wd:Q214642 .
 
    # this returns the labels
    ?s <http://schema.org/name> ?sname .
} 
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q2478977'), ('sname', 'Blue in the Face')]
[('s', 'http://www.wikidata.org/entity/Q2915460'), ('sname', 'Act of God')]
[('s', 'http://www.wikidata.org/entity/Q3213673'), ('sname', 'The Inner Life of Martin Frost')]
[('s', 'http://www.wikidata.org/entity/Q2364684'), ('sname', 'The Music of Chance')]
[('s', 'http://www.wikidata.org/entity/Q653447'), ('sname', 'Smoke')]
[('s', 'http://www.wikidata.org/entity/Q1877215'), ('sname', 'Lulu on the Bridge')]
[('s', 'http://www.wikidata.org/entity/Q3220942'), ('sname', 'The Center of the World')]


7

### 4. How many films were derived from the books of these two authors?

Let's find the books of the two authors and search for some propertes related to a movie.

In [ ]:
  queryString = """
SELECT ?s ?sname WHERE { 
        ?s wdt:P50 ?authors .
        
    # this returns the labels
    ?s <http://schema.org/name> ?sname .
    
    FILTER ((?authors = wd:Q214642) || (?authors = wd:Q190379)) .
} 
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1951470'), ('sname', 'Mr. Vertigo')]
[('s', 'http://www.wikidata.org/entity/Q492991'), ('sname', 'Timbuktu')]
[('s', 'http://www.wikidata.org/entity/Q581485'), ('sname', 'The Book of Illusions')]
[('s', 'http://www.wikidata.org/entity/Q526792'), ('sname', 'Travels in the Scriptorium')]
[('s', 'http://www.wikidata.org/entity/Q1723053'), ('sname', 'Saturday')]
[('s', 'http://www.wikidata.org/entity/Q1817920'), ('sname', 'Man in the Dark')]
[('s', 'http://www.wikidata.org/entity/Q1322083'), ('sname', 'Sunset Park')]
[('s', 'http://www.wikidata.org/entity/Q1584652'), ('sname', 'Invisible')]
[('s', 'http://www.wikidata.org/entity/Q28846426'), ('sname', '4 3 2 1')]
[('s', 'http://www.wikidata.org/entity/Q60412205'), ('sname', 'Sunset Park')]
[('s', 'http://www.wikidata.org/entity/Q16386002'), ('sname', 'Here and Now: Letters, 2008-2011')]
[('s', 'http://www.wikidata.org/entity/Q58883639'), ('sname', 'Saturday')]
[('s', 'http://www.wikidata.org/

53

Let's search the properties of the movie, to see how it's categorized. 

In [ ]:
  queryString = """
SELECT ?pname ?o ?oname WHERE { 
        wd:Q7348342 ?p ?o .
        
    # this returns the labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .    
    
} 
"""

print("Results")
run_query(queryString)

Results
[('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q44380'), ('oname', 'Cameron Diaz')]
[('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q214475'), ('oname', 'Ayaan Hirsi Ali')]
[('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q25089'), ('oname', 'Woody Allen')]
[('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q296047'), ('oname', 'Sam Harris')]
[('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q44461'), ('oname', 'Richard Dawkins')]
[('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q11424'), ('oname', 'film')]
[('pname', 'original language of film or TV show'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('oname', 'English')]
[('pname', 'color'), ('o', 'http://www.wikidata.org/entity/Q22006653'), ('oname', 'color')]
[('pname', 'country of origin'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oname', 'United States of America')]
[('pname', 'distribution format'), ('o', 'http://www.

19

Let's observe the properties 

In [19]:
  queryString = """
SELECT ?book ?author ?movie  WHERE { 
        ?b wdt:P50 ?a ;
            wdt:P4969 ?m .
        ?m wdt:P31 wd:Q11424 .
        
    # this returns the labels
    ?b <http://schema.org/name> ?book .
    ?a <http://schema.org/name> ?author .
    ?m <http://schema.org/name> ?movie .
    
    FILTER ((?a = wd:Q214642) || (?a = wd:Q190379)) .
} 
"""

print("Results")
run_query(queryString)


Results
[('book', 'Atonement'), ('author', 'Ian McEwan'), ('movie', 'Atonement')]
[('book', 'The Cement Garden'), ('author', 'Ian McEwan'), ('movie', 'The Cement Garden')]


2

### 5. Which author won more literature-related awards? 

Let's check some properties of them

In [ ]:
  queryString = """
SELECT ?p ?pname ?s ?sname WHERE { 
        ?a ?p ?s .
        
    # this returns the labels
    ?p <http://schema.org/name> ?pname .
    ?s <http://schema.org/name> ?sname .
    
    FILTER ((?a = wd:Q214642) || (?a = wd:Q190379)) .
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('pname', 'spouse'), ('s', 'http://www.wikidata.org/entity/Q1280035'), ('sname', 'Lydia Davis')]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pname', 'notable work'), ('s', 'http://www.wikidata.org/entity/Q581485'), ('sname', 'The Book of Illusions')]
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('pname', 'family name'), ('s', 'http://www.wikidata.org/entity/Q37565995'), ('sname', 'McEwan')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at'), ('s', 'http://www.wikidata.org/entity/Q1045828'), ('sname', 'University of East Anglia')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('pname', 'given name'), ('s', 'http://www.wikidata.org/entity/Q18336315'), ('sname', 'Ian')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pname', 'educated at'), ('s', 'http://www.wikidata.org/entity/Q1161297'), ('sname', 'University of Sussex')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pn

98

P166 is what I am searching for

In [33]:
  queryString = """
SELECT (COUNT (?o) as ?prizes) ?aname WHERE { 
        ?a wdt:P166 ?o .
        
        # this returns the labels
    ?a <http://schema.org/name> ?aname .
    
    FILTER ((?a = wd:Q214642) || (?a = wd:Q190379)) .
} 
GROUP BY ?aname
"""

print("Results")
run_query(queryString)

Results
[('prizes', '5'), ('aname', 'Paul Auster')]
[('prizes', '15'), ('aname', 'Ian McEwan')]


2

Have they ever being nominated for a Nobel award?

I need to use P1411

In [35]:
  queryString = """
SELECT ?o ?oname WHERE { 
        ?a wdt:P1411 ?o .
        
        # this returns the labels
    ?o <http://schema.org/name> ?oname .
    
    FILTER ((?a = wd:Q214642) || (?a = wd:Q190379)) .
} 
GROUP BY ?aname
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q160082'), ('oname', 'Booker Prize')]
[('o', 'http://www.wikidata.org/entity/Q2052291'), ('oname', 'International Booker Prize')]
[('o', 'http://www.wikidata.org/entity/Q1362723'), ('oname', 'International IMPAC Dublin Literary Award')]
[('o', 'http://www.wikidata.org/entity/Q628637'), ('oname', "European Film Award – People's Choice Award for Best European Film")]


4

No, neither.